In [1]:
!pip install requests

In [5]:
import requests

# Definisci l'URL dell'endpoint
base_url = "http://localhost:8080/api/accommodations/availableAccommodations"

# Parametri della richiesta
params = {
    "city": "Rome",       # Cambia con la città desiderata
    "guestSize": 2,       # Numero minimo di ospiti
    "startDate": "2025-09-01",  # Data di inizio (formato YYYY-MM-DD)
    "endDate": "2025-09-10",    # Data di fine (formato YYYY-MM-DD)
    "pageNumber": 0        # Numero della pagina (per la paginazione)
}

try:
    # Effettua la richiesta GET
    response = requests.get(base_url, params=params)
    
    # Controlla lo stato della risposta
    if response.status_code == 200:
        accommodations = response.json()
        print("Accommodations found:")
        for acc in accommodations:
            print(acc)
    else:
        print(f"Error {response.status_code}: {response.text}")

except requests.ConnectionError:
    print("Error: Unable to connect to the server. Is it running?")
except Exception as e:
    print(f"An error occurred: {e}")

Accommodations found:
{'accommodationId': '67ae3346cbd2cc39b74e5fe5', 'description': 'Appartamento con ogni confort, 7 min. dal Vaticano', 'type': 'Entire apartment', 'city': 'Rome', 'hostUsername': 'annagonzales', 'costPerNight': 110, 'averageRate': 5.0, 'photos': ['https://picsum.photos/200/300?random=41729']}
{'accommodationId': '67ae3346cbd2cc39b74e24c2', 'description': 'Sweet Home Colosseo', 'type': 'Entire apartment', 'city': 'Rome', 'hostUsername': 'michael77', 'costPerNight': 120, 'averageRate': 5.0, 'photos': ['https://picsum.photos/200/300?random=57480']}
{'accommodationId': '67ae3346cbd2cc39b74e5b16', 'description': 'La Dolce House', 'type': 'Entire apartment', 'city': 'Rome', 'hostUsername': 'lauren07', 'costPerNight': 48, 'averageRate': 5.0, 'photos': ['https://picsum.photos/200/300?random=9503']}
{'accommodationId': '67ae3346cbd2cc39b74ea595', 'description': 'Splendido Appartamento ROMA 9km da San PIETRO', 'type': 'Entire apartment', 'city': 'Rome', 'hostUsername': 'mossw

In [9]:
import requests

# URL di base per le API delle prenotazioni
base_url = "http://localhost:8080/api/books"
accommodation_id = "67ae3346cbd2cc39b74e9d97"  # ID della casa da bloccare

# Endpoint per il lock della casa con il giusto formato
lock_url = f"{base_url}/{{username}}/{accommodation_id}/lock"
book_url = f"{base_url}/{{username}}/{accommodation_id}/newBook"  # URL per la prenotazione



# Credenziali di login per due utenti diversi
users = [
    {"username": "sabrina34", "password": "largeScale"},
    {"username": "kathrynhorton", "password": "largeScale"}
]

# Dati della prenotazione (stessi per entrambi gli utenti)
booking_data = {
    "occupiedDates": [
        {
            "start": "2025-09-01",
            "end": "2025-09-10"
        }
    ],
    "guestFirstNames": ["Todd", "Parker"],
    "guestLastNames": ["Ellison", "Williams"]
}



# Sessioni per ciascun utente
sessions = {}

# 1️⃣ Login per entrambi gli utenti
for user in users:
    session = requests.Session()
    
    login_response = session.post("http://localhost:8080/api/users/login", json=user)
    
    if login_response.status_code == 200:
        sessions[user["username"]] = session
        print(f"✅ {user['username']} logged in successfully.")
    else:
        print(f"❌ {user['username']} login failed. Error: {login_response.text}")


# 3️⃣ Entrambi gli utenti provano a bloccare la stessa casa
if len(sessions) == 2:
    for user in users:
        username = user["username"]
        session = sessions[username]

        # Usa l'URL corretto per il lock con i parametri startDate e endDate
        user_lock_url = lock_url.format(username=username, accommodation_id=accommodation_id)
        lock_response = session.post(user_lock_url, params={"startDate": "2025-09-01", "endDate": "2025-09-10"})

        if lock_response.status_code == 200:
            print(f"🏠 {username} successfully locked the house!")
        else:
            print(f"🚫 {username} failed to lock the house. Server response: {lock_response.text}")

# 4️⃣ Entrambi gli utenti provano a prenotare la stessa casa
if len(sessions) == 2:
    for user in users:
        username = user["username"]
        session = sessions[username]

        # Usa l'URL corretto per il booking
        user_book_url = book_url.format(username=username, accommodation_id=accommodation_id)
        book_response = session.put(user_book_url, json=booking_data)

        if book_response.status_code == 200:
            print(f"📅 {username} successfully booked the house!")
        else:
            print(f"🚫 {username} failed to book the house. Server response: {book_response.text}")

# 5️⃣ Entrambi gli utenti visualizzano le loro pendingBooks
if len(sessions) == 2:
    for user in users:
        username = user["username"]
        session = sessions[username]

        pending_books_url = f"{base_url}/{username}/pendingBooks"
        pending_response = session.get(pending_books_url)

        if pending_response.status_code == 200:
            print(f"📖 {username}'s pending books: {pending_response.json()}")
        elif pending_response.status_code == 204:
            print(f"📖 {username} has no pending books.")
        else:
            print(f"🚫 {username} failed to fetch pending books. Server response: {pending_response.text}")


✅ sabrina34 logged in successfully.
✅ kathrynhorton logged in successfully.
🏠 sabrina34 successfully locked the house!
🚫 kathrynhorton failed to lock the house. Server response: Accommodation already booked from another user or time period not valid
📅 sabrina34 successfully booked the house!
🚫 kathrynhorton failed to book the house. Server response: Internal Server Error: Error occurred while adding the booking: Accommodation 67ae3346cbd2cc39b74e9d97 is not available for the selected period.
📖 sabrina34's pending books: [{'occupiedDates': [{'start': '2025-09-01', 'end': '2025-09-10'}], 'username': 'sabrina34', 'email': 'itran@example.org', 'birthPlace': 'Krystalfort', 'address': '7973 Kendra Highway', 'addressNumber': 16, 'birthDate': '1975-10-26', 'guestFirstNames': ['Todd', 'Parker'], 'guestLastNames': ['Ellison', 'Williams'], 'bookDate': '2025-02-14', 'accommodationId': '67ae3346cbd2cc39b74e8c4b', 'endDate': '2025-09-10', 'startDate': '2025-09-01'}, {'occupiedDates': [{'start': '202